In [5]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
for df in [train_df, test_df]:
    df["date * trainNo"] = df["date"].astype("str") + " * " + df["trainNo"]

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488885 entries, 0 to 1488884
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              1488885 non-null  int64  
 1   date            1488885 non-null  int64  
 2   lineName        1488885 non-null  object 
 3   directionCode   1488885 non-null  int64  
 4   trainNo         1488885 non-null  object 
 5   stopStation     1488885 non-null  object 
 6   planArrival     1488885 non-null  object 
 7   delayTime       1488885 non-null  float64
 8   date * trainNo  1488885 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 102.2+ MB


In [ ]:
train_date = train_df.groupby("date").count()[["id"]].sort_index().reset_index()
test_date = test_df.groupby("date").agg({"id":"count", "target":"sum"})

date_df = train_date.merge(test_date, on="date", how="outer" , suffixes=["_train", "_test"]).sort_values("date")

def func(x):
    is_train = x["id_train"] == x["id_train"]
    is_test = x["id_test"] == x["id_test"]
    is_train_test = is_train and is_test
    if is_train_test:
        return "train_test"
    elif is_train:
        return "train"
    elif is_test:
        return "test"
    else:
        assert("error")

date_df["appearance"] = date_df.apply(func, axis=1)
# trainにしかいない日、testにしかいない日、両方にいる日、がランダムに分かれているっぽい
# trainかつtestの日はflagがついてるか確認するべき
# trainかつtestの日がtrainのデータが少ない訳でもない?
# どの線のどの時間帯のどの電車がtrain, testにあるのかは見た方が良さそう
# testのうちだいたい10%くらいにflagが立っていて、それはtrain_testでもtest_onlyでも変わらない
# ので、validの分割は、train_testの日から半分or日単位と、train_onlyの日から日単位で分ける
# cvの切り方を乱数で変えることでcv安定させたい
# testで予測対象になっている日、最初の遅延時間がわかっているものもある
# だいたい10%くらいでdate * trainNoで最初の遅延時刻が得られるものがある

In [ ]:
date_df

In [ ]:
test_g = test_df.groupby(["date", "trainNo"]).agg({"id":"count", "target":"sum"})
test_g[:100]

In [ ]:
print(test_df["date * trainNo"].nunique())
test_g["is_firstDelayTime"] = 0
for val in test_df["date * trainNo"].unique():
    temp = test_df[test_df["date * trainNo"] == val]
    tar = temp.iloc[0]["delayTime"]
    if tar == tar: # Nanじゃないなら
        date, trainNo = val.split(" * ")
        date = int(date)
        test_g["is_firstDelayTime"].loc[date, trainNo] = 1
        

In [ ]:
print(test_g.shape)
print(test_g.sum())

In [ ]:
train_df_d = pd.read_csv("../input/decoded_train.csv", encoding="cp932")
test_df_d = pd.read_csv("../input/decoded_test.csv", encoding="cp932")

In [ ]:
train_g = train_df_d.groupby(["date", "trainNo"]).agg({"id":"count"})


In [ ]:
train_g[train_g["id"] > 50]

In [ ]:
train_df_d[(train_df_d["date"] == 20201013) & (train_df_d["trainNo"] == "UFiyVP")]

In [2]:
import pandas as pd
train = pd.read_csv("../input/train.csv", usecols=["id"], dtype={"id":"int64"})

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1488885 entries, 0 to 1488884
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   id      1488885 non-null  int64
dtypes: int64(1)
memory usage: 11.4 MB
